In [2]:
from urllib import parse as url
from bs4 import BeautifulSoup
import pandas as pd
import requests
import functools

from urllib.request import Request, urlopen

In [4]:
def steam_search_games(term):
  search_url = 'https://store.steampowered.com/search/?term=' + url.quote(term.strip()) + '&ignore_preferences=1'
  page = requests.get(search_url)

  soup = BeautifulSoup(page.text)
  games_list = soup.find_all(id='search_resultsRows')[0]

  results = []
  
  for row_link in games_list.find_all('a'):
    app_id = row_link.get('data-ds-appid')
    name = row_link.find_all(class_='title')[0].string
    results.append(dict(app_id=app_id, name=name))

  return results

In [5]:
print(steam_search_games('neon abyss'))

[{'app_id': '788100', 'name': 'Neon Abyss'}, {'app_id': '1518790', 'name': 'Neon Abyss - Alter Ego'}, {'app_id': '1262000', 'name': 'Neon Abyss - Lovable Rogues Pack'}, {'app_id': '1396440', 'name': 'Neon Abyss Soundtrack'}, {'app_id': None, 'name': 'Neon Abyss Deluxe Edition'}, {'app_id': '1182150', 'name': 'Neon Abyss Demo'}, {'app_id': None, 'name': 'The Neon Cloud Runner Bundle'}, {'app_id': None, 'name': 'Tesla Ultimate Bundle'}, {'app_id': '1048350', 'name': 'Nigate Tale'}, {'app_id': None, 'name': '10tons Total'}, {'app_id': None, 'name': 'Roguelike Bundle'}, {'app_id': '1276781', 'name': 'Tidal Shock: DIVE CREW DLC'}, {'app_id': '1276780', 'name': 'Tidal Shock: SURFERS DLC'}, {'app_id': '847810', 'name': 'Endless Horizon'}, {'app_id': '873720', 'name': 'Pac Adventures 3D'}]


In [36]:
def steam_db_price_history(app_id):
  price_url = 'https://steamdb.info/api/GetPriceHistory/?appid=' + str(app_id).strip() + '&cc=eu'

  headers={
    'accept-language': 'en-US;q=0.9,en;q=0.8',
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36',
    'x-requested-with': 'XMLHttpRequest',
    'accept-encoding': 'gzip, deflate, br',
    # ':authority': 'steamdb.info',
    # ':method': 'GET',
    # ':path': f'/api/GetPriceHistory/?appid={app_id}&cc=eu',
    # ':scheme': 'https',
    'referer': f'https://steamdb.info/app/{app_id}/',
  }

  prices_res = requests.get(price_url, headers=headers, timeout=30)
  prices_json = prices_res.json()
  
  history = prices_json['data']['history']
  price_history = [ dict(time=h['x'], price=h['y'], discount=h['d'] / 100) for h in history ]
  
  return price_history

In [37]:
def steam_db_discount_data(price_history):
    discount_history = [ datapoint for datapoint in price_history if datapoint['price'] > 0 and datapoint['discount'] > 0 ]

    if len(discount_history) > 0:
        latest = discount_history[len(discount_history) - 1]

        biggest = functools.reduce(lambda a, b: a if a['discount'] > b['discount'] else b, discount_history)

        same = (latest['time'] == biggest['time'])

        return dict(latest=latest, biggest=biggest, same=same)
    else:
        return None

In [38]:
def ig_search_games(term):
  region = ''
  query = url.quote(term.strip())
  currency = 'EUR'

  search_url = 'https://www.instant-gaming.com/en/search/?' \
    + f'all_types=1&all_cats=1&min_price=0&max_price=100&noprice=1&min_discount=0&max_discount=100&min_reviewsavg=10&max_reviewsavg=100&currency={currency}' \
    + f'&noreviews=1&available_in={region}&gametype=all&sort_by=&query={query}'

  headers = {
    'accept-language': 'en;q=0.8',
    'user-agent': 'Python/3.x requests/1.x',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
  }

  page = requests.get(search_url, headers=headers)
  soup = BeautifulSoup(page.text)

  games_list = soup.find_all('div', class_='item')
  results = []

  for item in games_list:
    name = item.find('div', class_='name').string
    price = item.get('data-price')

    discount_item = item.find('div', class_='discount')
    discount_string = discount_item.string if discount_item is not None else None
    discount = float(discount_string.replace('%', '').replace('-', '')) / 100 if discount_string is not None else None

    platform_badge_classes = item.find('div', class_='badge').get('class')
    platform = platform_badge_classes[1] if len(platform_badge_classes) >= 2 else 'unknown'

    dlc = True if str(item.get('data-dlc')) == '1' else False
    link = item.find('a', class_='cover').get('href')

    results.append(dict(name=name, price=price, discount=discount, platform=platform, dlc=dlc, link=link))

  return results
  # return pd.DataFrame(results, columns=['name', 'price', 'discount', 'platform', 'dlc', 'link'])

In [39]:
def steam_user_wishlist(vanity_url):
    done = False
    page = 0
    result_list = []

    def extract_data(data):
        sub = data['subs'][0] if len(data['subs']) > 0 else None
        early_access = data.get('early_access')
        prerelease = data.get('prerelease')

        return dict(
            name=data['name'],
            priority=data['priority'],
            price=sub['price'] / 100 if sub else None,
            discount=sub['discount_pct'] / 100 if sub else None,
            early_access=True if early_access is not None and early_access > 0 else False,
            prerelease=True if prerelease is not None and prerelease > 0 else False,
        )

    while not done:
        url = f'https://store.steampowered.com/wishlist/id/{vanity_url}/wishlistdata/?p={page}'
        parsed_json = requests.get(url).json()

        if len(parsed_json) > 0:
            result_list += [ dict(app_id=app_id, **extract_data(data)) for app_id, data in parsed_json.items() ]
            page += 1
        else:
            done = True

    result_list.sort(key=lambda x: int(x['priority']))
    # return pd.DataFrame(result_list)
    return result_list

In [40]:
def enhanced_wishlist_data(vanity_url, num=10):
    wishlist = steam_user_wishlist(vanity_url)[:num]
    enhanced_wishlist = []

    for wishlist_item in wishlist:
        ig_results = ig_search_games(wishlist_item['name'])
        ig_best = ig_results[0] if len(ig_results) > 0 else None
        ig_data = dict(search_results=ig_results, best_result=ig_best)

        price_history = steam_db_price_history(wishlist_item['app_id'])
        sales_data = steam_db_discount_data(price_history)

        row = dict(**wishlist_item, price_history=price_history, ig_data=ig_data, sales_data=sales_data)
        enhanced_wishlist.append(row)

    return enhanced_wishlist


In [41]:
# steam_search_games('prey')
# [ x for x in ig_search_games('fallout 4') if not x['dlc'] ]

TypeError: 'int' object is not iterable

In [42]:
ew = enhanced_wishlist_data('jackss14')

In [43]:
ew

[{'app_id': '526870',
  'name': 'Satisfactory',
  'priority': 1,
  'price': 29.99,
  'discount': 0.0,
  'early_access': True,
  'prerelease': False,
  'price_history': [{'time': 1591636144000, 'price': 26.99, 'discount': 0.1},
   {'time': 1594316604000, 'price': 29.99, 'discount': 0.0},
   {'time': 1600968684000, 'price': 23.99, 'discount': 0.2},
   {'time': 1601313660000, 'price': 29.99, 'discount': 0.0},
   {'time': 1603991003000, 'price': 23.99, 'discount': 0.2},
   {'time': 1604345100000, 'price': 29.99, 'discount': 0.0},
   {'time': 1606327649000, 'price': 23.99, 'discount': 0.2},
   {'time': 1607105190000, 'price': 29.99, 'discount': 0.0},
   {'time': 1608660844000, 'price': 23.99, 'discount': 0.2},
   {'time': 1609875725000, 'price': 29.99, 'discount': 0.0},
   {'time': 1613066850000, 'price': 23.99, 'discount': 0.2},
   {'time': 1613417192000, 'price': 29.99, 'discount': 0.0},
   {'time': 1618334147000, 'price': 23.99, 'discount': 0.2},
   {'time': 1618765805000, 'price': 29.99